## Laboratorio Final - Equipo Rocket

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from collections import Counter
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt
import warnings
warnings.filterwarnings("ignore")
from collections import Counter

In [2]:
#Cargamos los datos, y eliminamos los que no tienen valor
df = pd.read_csv("calendario_economico.csv") 
df.dropna(inplace = True)

In [3]:
data2=df.loc[df['Name'] == 'Retail Sales (MoM)']
RetailSales=data2.loc[df['Country'] == 'European Monetary Union']
RetailSales=RetailSales.reset_index(drop=True)

In [4]:
RetailSales['timestamp']=pd.to_datetime(RetailSales['timestamp'])
RetailSales['timestamp']=RetailSales['timestamp'].dt.tz_localize('UTC')

In [5]:
#definimos escenarios por restricciones
def escenario(x):
    x['escenario']= x['actual']
    for i in range(len(x)):
        if x['actual'][i] >= x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i] = "A"
        elif x['actual'][i] >= x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="B"
        elif x['actual'][i] < x['consensus'][i] >= x['previous'][i]:
            x['escenario'][i]="C"
        elif x['actual'][i] < x['consensus'][i] < x['previous'][i]:
            x['escenario'][i]="D"
    return x
    

In [6]:
# utilizamos la función escenario para obtener cada comunicado del indicador económico si es A, B, C ó D dependiendo del actual,
# previous y consensus.
RS=escenario(RetailSales)

In [7]:
# Cargamos los datos de los precios del Euro-Dolar
df = pd.read_csv("precios_historicos_eurusd.csv") 
df['timestamp']=pd.to_datetime(df['timestamp'])
df['timestamp']=df['timestamp'].dt.tz_localize('UTC')

In [8]:
# Checando valores en precios EUR/USD con misma hora que nuestro calendario económico
# Data frame con los valores que coinciden las horas
comparacionRS=df[df.timestamp.isin(RS.timestamp)]

In [9]:
# Utilizamos compresión de listas para sacar un DataFrame de los indicadores para hacer match entre los comunicados del
# indicador y los precios
#Se busca desde la fecha y hora del indicador agarrar los precios de 30 minutos antes y 30 minutos despues (mas menos 6 periodos)
periodosRS = [df.iloc[i-6:i+7]for i in comparacionRS.index]

In [29]:
# Con esta función se saca dirección, pips alcistas, pips bajistas y volatilidad de los precios de cada comunicado del indicador
# Se le mete el DataFrame de los comunicados de nuestro indicador y el match de los precios con el indicador
def finalDF(DataFrame,periodos):

    DataFrame['direccion']= DataFrame['actual']
    DataFrame['pips alcistas']=DataFrame['actual']
    DataFrame['pips bajistas']= DataFrame['actual']
    DataFrame['volatilidad']=DataFrame['actual']

    for i in range(len(DataFrame)):
        if periodos[i]["close"].iloc[-1] > periodos[i]["open"].iloc[6]:
            DataFrame['direccion'][i] = 1
        else:
            DataFrame['direccion'][i] = -1
                
        DataFrame['pips alcistas'][i] = periodos[i]["high"].iloc[6:-1].max() - periodos[i]["open"].iloc[0]
        DataFrame['pips bajistas'][i] = periodos[i]["open"].iloc[0] - periodos[i]["low"].iloc[6:-1].min()
        DataFrame['volatilidad'][i] = periodos[i]["high"].max()-periodos[i]["low"].min()
    return DataFrame




In [30]:
# utilizamos la funcion finalDF() para obtener nuestro DataFrame final en retail sales.
FinalDFRS = finalDF(RS,periodosRS)
FinalDFRS

,timestamp,Currency,Name,Country,Volatility,actual,previous,consensus,escenario,direccion,pips alcistas,pips bajistas,volatilidad
0,2009-01-09 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,0.32,0.30,-0.1,B,1.0,0.00537,-0.00138,0.00642
1,2009-02-04 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-0.90,0.32,-0.2,D,1.0,0.00459,-0.00121,0.00547
2,2009-03-13 10:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,0.10,-0.90,0.2,C,-1.0,-0.00066,0.00418,0.00460
3,2009-04-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-1.00,0.10,-0.3,D,-1.0,-0.00080,0.00354,0.00490
4,2009-05-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,2,-0.30,-1.00,0.1,C,1.0,0.00088,0.00150,0.00266
...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2019-05-06 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,0.00,0.40,-0.1,B,1.0,-0.00006,0.00078,0.00081
125,2019-06-05 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,-0.40,0.00,-0.4,B,1.0,0.00137,-0.00031,0.00170
126,2019-07-04 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,-0.30,-0.40,0.3,C,1.0,-0.00010,0.00092,0.00100
127,2019-08-02 09:00:00+00:00,EUR,Retail Sales (MoM),European Monetary Union,1,1.10,-0.30,0.2,A,1.0,0.00050,0.00018,0.00074


In [12]:
#Usamos la funcion Counter para contar cuantas veces se repiten las palabras
newcounter = Counter(FinalDFRS['escenario'])
escenariosRS = pd.DataFrame.from_dict(newcounter, orient='index').reset_index()
escenariosRS.columns = ['escenario', 'cantidad']
escenariosRS
###----------------cuantos escenarios hubo en cada uno A B C y D (RS)-------------------------

,escenario,cantidad
0,B,37
1,D,24
2,C,37
3,A,31


In [13]:
# Sacando cuales informes pertenecen dependiendo de escenario A, B, y C para checar comportamiento
def getescenario(DF):
    escA=DF[DF.escenario.isin(["A"])]
    escB=DF[DF.escenario.isin(["B"])]
    escC=DF[DF.escenario.isin(["C"])]
    escD=DF[DF.escenario.isin(["D"])]
    return escA,escB,escC,escD



In [14]:
#para sacar el escenario "A"=RSescenarios[0]
#para sacar el escenario B =RSescenarios[1]
#para sacar el escenario C =RSescenarios[2]
#para sacar el escenario D =RSescenarios[3]
RSescenarios=getescenario(FinalDFRS)